In [1]:
import auxiliary as aux
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from lightgbm import LGBMRegressor

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import mean_squared_error

from sklearn.cluster import k_means

from copy import deepcopy
from tqdm import tqdm

In [2]:
aux.data.head()

,F_1_0,F_1_1,F_1_2,F_1_3,F_1_4,F_1_5,F_1_6,F_1_7,F_1_8,F_1_9,...,F_4_5,F_4_6,F_4_7,F_4_8,F_4_9,F_4_10,F_4_11,F_4_12,F_4_13,F_4_14
row_id,,,,,,,,,,,,,,,,,,,,,
0,-0.354591,-0.464038,2.304115,0.734486,1.696395,0.136285,-0.518344,0.502640,-1.852504,-0.500665,...,3.744152,0.794438,0.265185,-0.561809,0.196480,0.373434,6.206995,3.809505,1.236486,1.182055
1,1.380940,-0.499626,-0.418548,1.911725,-0.826130,-1.715371,-0.577091,-1.041486,0.596067,-0.363425,...,-2.895826,-0.738275,2.361818,-0.060753,0.727249,-0.271882,5.232157,-4.218259,-2.724883,-0.063775
2,0.256023,-1.059874,NaN,0.345678,1.513814,1.243864,-0.509648,-0.800481,-0.115945,0.595777,...,2.252834,0.472496,2.491386,0.353381,-0.260682,-0.000833,-0.116457,-2.131747,3.661499,-0.131576
3,-0.728420,-2.432399,-2.453602,-0.020509,0.333397,0.086049,-1.787601,0.667011,0.761564,-2.217847,...,2.004600,-4.664806,-0.847211,-0.264249,0.664334,-0.557868,8.499483,-4.738799,-3.054611,0.494152
4,0.590212,-0.066127,0.468009,-1.096038,0.119399,-1.809710,0.466358,-0.053196,-0.580320,-1.143500,...,0.976937,2.558883,3.377724,0.846891,0.696032,0.554121,-5.979714,-2.869631,3.733057,-0.722943


In [4]:
# def impute(df, pipe):
#     """
#     :param pipe - estimator or transformer pipeline
#     """
#     imp = df.copy()
#     metrics = []
#     # get columns with nans
#     nan_cols = df.columns[df.isna().any()]
#     is_estimator = hasattr(pipe, 'predict')
#     if not is_estimator:
#         pipe.fit(df[nan_cols])    # fit if pipe is transformer
#         # predict
#         stats = pd.DataFrame([pipe[-1].statistics_ if isinstance(pipe, Pipeline) else pipe.statistics_] * df.shape[0], index=df.index, columns=df.columns)

#     for col in tqdm(nan_cols):
#         nan_rows = df[col].isna()   # select rows that are NaN in this columns
#         y_train = df.loc[~nan_rows, col]

#         # fit if pipe is estimator
#         if is_estimator:
#             # train/test split
#             X_train = df[~nan_rows].drop(col, axis=1)
#             X_test = df[nan_rows].drop(col, axis=1)
#             pipe.fit(X_train, y_train)
#             # predict
#             train_pred = pipe.predict(X_train)
#             imp.loc[nan_rows, col] = pipe.predict(X_test)
#         else:
#             # predict if pipe is transformer
#             train_pred = stats.loc[~nan_rows, col]
#             imp.loc[nan_rows, col] = stats.loc[nan_rows, col].values
          
#         # score pipeline
#         score = np.sqrt(mean_squared_error(y_train, train_pred))    # RMSE
#     metrics.append(score)
#     print(f'\n{np.mean(metrics)}')
#     return imp

In [5]:
# def groupstat(df, *, gcol, func='mean'):
#     stats = df.groupby(gcol).transform(func)
#     if stats.isna().any().any():
#         stats.fillna(stats.agg(func), inplace=True)
#         print(f'Stats contain NaNs! Filled with {func}')
#     imp = df.fillna(stats)
#     # if imp.isna().any().any():
#     #     imp.fillna(imp.agg(func), inplace=True)
#     #     print('Data still contain NaNs! Select another group columns')

#     # compute score
#     metrics = []
#     nan_cols = df.columns[df.isna().any()]
#     for col in tqdm(nan_cols):
#         nan_rows = df[col].isna()   # select rows that are NaN in this columns
#         y_train = df.loc[~nan_rows, col]
#         train_pred = stats.loc[~nan_rows, col]
#         score = np.sqrt(mean_squared_error(y_train, train_pred))    # RMSE

#     metrics.append(score)
#     print(f'\n{np.mean(metrics)}')
#     return imp

## impute with mean

In [30]:
# subset 1: mean=0.9998099555723642; median=0.999810232349789
s1 = aux.ml_impute(aux.subset[1], SimpleImputer(strategy='mean'))

100%|██████████| 15/15 [00:00<00:00, 28.27it/s]


0.9998099555723642


In [31]:
# subset 3: mean=1.000103751834463; median=1.0001038164849292
s3 = aux.ml_impute(aux.subset[3], SimpleImputer(strategy='mean'))

100%|██████████| 25/25 [00:00<00:00, 30.67it/s]


1.000103751834463


In [18]:
# subset 4: mean=0.7763533590477227; median=0.7820259934486685
# s4 = aux.ml_impute(aux.subset[4], SimpleImputer(strategy='mean'))

## impute with groupstat

In [21]:
# subset 1
np.random.seed(19)
# columns = np.random.choice(subset[2].columns, size=1, replace=False).tolist()
columns = ['F_2_2']
print(columns)
s1 = aux.groupstat(pd.concat([aux.subset[1], aux.subset[2]], axis=1), gcol=columns, func='mean').drop(aux.subset[2].columns, axis=1)
# ['F_2_16', 'F_2_21', 'F_2_19', 'F_2_13', 'F_2_24']

['F_2_2']


100%|██████████| 15/15 [00:00<00:00, 36.51it/s]


0.9998041082081748


In [45]:
# subset 3
np.random.seed(17)
# columns = np.random.choice(subset[2].columns, size=2, replace=False).tolist()
columns = ['F_2_24', 'F_2_10', 'F_2_2']
print(columns)
s3 = aux.groupstat(pd.concat([aux.subset[3], aux.subset[2]], axis=1), gcol=columns, func='mean').drop(aux.subset[2].columns, axis=1)

['F_2_24', 'F_2_10', 'F_2_2']
Stats contain NaNs! Filled with mean


100%|██████████| 25/25 [00:00<00:00, 35.67it/s]


0.9995054012954069


## impute with estimator

In [37]:
# subset 1
estimator = LGBMRegressor(
    random_state=7, n_jobs=-1,
    n_estimators=100,
    # max_depth=-1,
    num_leaves=23
)
s1 = aux.ml_impute(aux.subset[1], estimator)

100%|██████████| 15/15 [00:42<00:00,  2.85s/it]


0.9977684242961092


In [ ]:
# subset 3
estimator = LGBMRegressor(
    random_state=7, n_jobs=-1,
    n_estimators=25,
    max_depth=-1
)
s3 = aux.ml_impute(aux.subset[3], estimator)

In [9]:
# subset 4
estimator = LGBMRegressor(
    random_state=7, n_jobs=-1,
    n_estimators=1500,
    max_depth=-1,
    num_leaves=29
)
s4 = aux.ml_impute(aux.subset[4], estimator)

100%|██████████| 15/15 [16:58<00:00, 67.90s/it]


0.18989910477455568


## concat

In [47]:
predicted = pd.concat([s1, aux.subset[2], s3, s4], axis=1)

## full impute

In [ ]:
# FULL DATA GROUPSTAT - kaggle 1.44
np.random.seed(19)
columns = np.random.choice(aux.subset[2].columns, 3).tolist()
print(columns)

predicted = aux.groupstat(aux.data, gcol=columns, func='mean')

## submission

In [ ]:
aux.save_submission(predicted)

In [51]:
#